## 這份檔案是基於 ```test.ipynb```，只是刪除部分細項

上次的 ```test.ipynb``` 讀取的是所有資料  
這份檔案則是要連結到 demo account

到 [新的關於 demo account 可用的 API 網頁(https://developer.oanda.com/rest-live-v20/account-ep/)  
並修改下方的 url 變數

In [4]:
import requests
import defs

In [6]:
# 如果要找 500 筆資料，每次都跟網站 request 一次只會讓速度變慢
# 因此，可以用 requests 裡的 Session 物件
session = requests.Session()

到 [ONNDA API 的網站查看他們的規定](https://developer.oanda.com/rest-live-v20/instrument-ep/)  
發現 Instrument Endpoints 的規則是:  
https://api-fxtrade.oanda.com/v3/ + /instruments/instrument (客製化)/candles + .....

In [8]:
url = f'{defs.OANDA_URL}accounts/{defs.ACCOUNT_ID}/instruments'

In [10]:
url

'https://api-fxpractice.oanda.com/v3/accounts/101-001-20505319-001/instruments'

In [14]:
response = session.get(url, params=None, headers=defs.SECURE_HEADER)

In [16]:
response.status_code # 成功取得資料 (200 的定義可以在 OANDA API 網站上查閱)

200

In [22]:
data = response.json()

In [25]:
type(data)

dict

In [28]:
data.keys() # we want the instruments key

dict_keys(['instruments', 'lastTransactionID'])

In [61]:
instruments = data['instruments']
print(type(instruments)) # a dict
len(instruments) # Demo account 可以交易 68 種商品

<class 'list'>


68

In [64]:
print(instruments[0])
type(instruments[0]) # still a dict

{'name': 'ZAR_JPY', 'type': 'CURRENCY', 'displayName': 'ZAR/JPY', 'pipLocation': -2, 'displayPrecision': 3, 'tradeUnitsPrecision': 0, 'minimumTradeSize': '1', 'maximumTrailingStopDistance': '100.000', 'minimumTrailingStopDistance': '0.005', 'maximumPositionSize': '0', 'maximumOrderUnits': '100000000', 'marginRate': '0.07', 'guaranteedStopLossOrderMode': 'DISABLED', 'tags': [{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}], 'financing': {'longRate': '0.0242', 'shortRate': '-0.0761', 'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1}, {'dayOfWeek': 'TUESDAY', 'daysCharged': 1}, {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1}, {'dayOfWeek': 'THURSDAY', 'daysCharged': 1}, {'dayOfWeek': 'FRIDAY', 'daysCharged': 1}, {'dayOfWeek': 'SATURDAY', 'daysCharged': 0}, {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}}


dict

In [55]:
instruments[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [71]:
instrument_data =[] 
for item in instruments: # 將可交易的商品迴圈，並把各商品的名稱、margin rate 等存起來
    new_ob = dict(
        name = item['name'],
        type = item['type'],
        displayName = item['displayName'],
        pipLocation = item['pipLocation'],
        marginRate = item['marginRate']
    )
    instrument_data.append(new_ob) # list 裡面放入 dict

In [81]:
for item in instrument_data[0:3]:
    print(item)

{'name': 'ZAR_JPY', 'type': 'CURRENCY', 'displayName': 'ZAR/JPY', 'pipLocation': -2, 'marginRate': '0.07'}
{'name': 'EUR_HUF', 'type': 'CURRENCY', 'displayName': 'EUR/HUF', 'pipLocation': -2, 'marginRate': '0.05'}
{'name': 'EUR_DKK', 'type': 'CURRENCY', 'displayName': 'EUR/DKK', 'pipLocation': -4, 'marginRate': '0.1'}


## 將資料放入 data frame:

In [86]:
import pandas as pd

In [87]:
instrument_df = pd.DataFrame.from_dict(instrument_data)

In [88]:
instrument_df

,name,type,displayName,pipLocation,marginRate
0,ZAR_JPY,CURRENCY,ZAR/JPY,-2,0.07
1,EUR_HUF,CURRENCY,EUR/HUF,-2,0.05
2,EUR_DKK,CURRENCY,EUR/DKK,-4,0.1
3,USD_MXN,CURRENCY,USD/MXN,-4,0.10
4,GBP_USD,CURRENCY,GBP/USD,-4,0.03
...,...,...,...,...,...
63,USD_PLN,CURRENCY,USD/PLN,-4,0.05
64,CAD_HKD,CURRENCY,CAD/HKD,-4,0.1
65,GBP_CAD,CURRENCY,GBP/CAD,-4,0.03
66,GBP_PLN,CURRENCY,GBP/PLN,-4,0.03


In [89]:
instrument_df.to_pickle("instruments.pkl")
# pkl: binary bits format provided by pandas

## 存入檔案後，之後就不用每次都向 OANDA 官網 request 資訊了

In [91]:
new_table = pd.read_pickle('instruments.pkl')

In [92]:
new_table

,name,type,displayName,pipLocation,marginRate
0,ZAR_JPY,CURRENCY,ZAR/JPY,-2,0.07
1,EUR_HUF,CURRENCY,EUR/HUF,-2,0.05
2,EUR_DKK,CURRENCY,EUR/DKK,-4,0.1
3,USD_MXN,CURRENCY,USD/MXN,-4,0.10
4,GBP_USD,CURRENCY,GBP/USD,-4,0.03
...,...,...,...,...,...
63,USD_PLN,CURRENCY,USD/PLN,-4,0.05
64,CAD_HKD,CURRENCY,CAD/HKD,-4,0.1
65,GBP_CAD,CURRENCY,GBP/CAD,-4,0.03
66,GBP_PLN,CURRENCY,GBP/PLN,-4,0.03
